In [2]:
print("hello")

hello


In [27]:
import os
import json

def load_homestays():
    with open('data/homestays.json', 'r', encoding='utf-8') as f:
        return json.load(f)

def check_and_rename_images(base_dir):
    homestays = load_homestays()
    homestay_names = {homestay['name'].lower(): homestay['id'] for homestay in homestays}

    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        folder_name_lower = folder_name.lower()
        if folder_name_lower in homestay_names:
            print(f"\n✅ Đã tìm thấy folder cho homestay: {folder_name}")
            image_files = [f for f in os.listdir(folder_path)
                           if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
            if not image_files:
                print(f"⚠️ Không tìm thấy ảnh trong folder {folder_name}")
                continue
            for i, old_name in enumerate(sorted(image_files), 1):
                old_path = os.path.join(folder_path, old_name)
                new_name = f"{i}.jpg"
                new_path = os.path.join(folder_path, new_name)
                try:
                    os.rename(old_path, new_path)
                    print(f"  Đã đổi tên: {old_name} -> {new_name}")
                except Exception as e:
                    print(f"  Lỗi khi đổi tên {old_name}: {str(e)}")
        else:
            print(f"\n❌ Không tìm thấy homestay tương ứng cho folder: {folder_name}")


In [29]:
# Thay đường dẫn này bằng đường dẫn thực tế tới folder của bạn
base_dir = r"C:\Users\Xpeedent\Desktop\Image for homestay\Image for homestay"
check_and_rename_images(base_dir)


✅ Đã tìm thấy folder cho homestay: 85B Homestay
  Đã đổi tên: 1.jpg -> 1.jpg
  Đã đổi tên: 2.jpg -> 2.jpg
  Đã đổi tên: 3.jpg -> 3.jpg
  Đã đổi tên: 4.jpg -> 4.jpg
  Đã đổi tên: 5.jpg -> 5.jpg
  Đã đổi tên: 6.jpg -> 6.jpg
  Đã đổi tên: 7.jpg -> 7.jpg
  Đã đổi tên: 8.jpg -> 8.jpg

✅ Đã tìm thấy folder cho homestay: Ace Homestay
  Đã đổi tên: 1.jpg -> 1.jpg
  Lỗi khi đổi tên 10.jpg: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Xpeedent\\Desktop\\Image for homestay\\Image for homestay\\Ace Homestay\\10.jpg' -> 'C:\\Users\\Xpeedent\\Desktop\\Image for homestay\\Image for homestay\\Ace Homestay\\2.jpg'
  Lỗi khi đổi tên 11.jpg: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Xpeedent\\Desktop\\Image for homestay\\Image for homestay\\Ace Homestay\\11.jpg' -> 'C:\\Users\\Xpeedent\\Desktop\\Image for homestay\\Image for homestay\\Ace Homestay\\3.jpg'
  Lỗi khi đổi tên 12.jpg: [WinError 183] Cannot create a file when that file al

In [30]:
import os
import json
import cloudinary
import cloudinary.uploader

# Cấu hình Cloudinary
cloudinary.config(
  cloud_name = 'drsfudaav',
  api_key = '281451562111515',
  api_secret = 'WcVlLOCmxmHTP_q2zVdKlNUyoxc'
)

def upload_images_and_update_json(base_dir, json_path):
    # Load homestays
    with open(json_path, 'r', encoding='utf-8') as f:
        homestays = json.load(f)

    # Tạo dict cho phép tra cứu nhanh
    name_to_homestay = {h['name'].lower(): h for h in homestays}

    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        folder_name_lower = folder_name.lower()
        if folder_name_lower in name_to_homestay:
            print(f"Uploading images for: {folder_name}")
            image_urls = []
            for img_file in sorted(os.listdir(folder_path)):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                    img_path = os.path.join(folder_path, img_file)
                    result = cloudinary.uploader.upload(img_path, folder=f"homestay/{folder_name}")
                    image_urls.append(result['secure_url'])
                    print(f"  Uploaded: {img_file} -> {result['secure_url']}")
            # Cập nhật vào homestay
            name_to_homestay[folder_name_lower]['image_urls'] = image_urls
        else:
            print(f"Không tìm thấy homestay tương ứng cho folder: {folder_name}")

    # Lưu lại file json
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(homestays, f, ensure_ascii=False, indent=2)
    print("Đã cập nhật xong homestays.json!")

if __name__ == "__main__":
    base_dir = r'C:\\Users\\Xpeedent\\Desktop\\Image for homestay\\Image for homestay\\Space Homestay'  # Đường dẫn folder ảnh
    json_path = 'data/homestays.json'  # Đường dẫn file json
    upload_images_and_update_json(base_dir, json_path)

Đã cập nhật xong homestays.json!


In [32]:
import os
import json
import cloudinary
import cloudinary.uploader

# Cấu hình Cloudinary
cloudinary.config(
  cloud_name = 'drsfudaav',
  api_key = '281451562111515',
  api_secret = 'WcVlLOCmxmHTP_q2zVdKlNUyoxc'
)

def upload_images_for_homestay(homestay_name, image_dir, json_path):
    # Load homestays
    with open(json_path, 'r', encoding='utf-8') as f:
        homestays = json.load(f)

    # Tìm homestay theo tên (không phân biệt hoa thường)
    homestay = next((h for h in homestays if h['name'].lower() == homestay_name.lower()), None)
    if not homestay:
        print(f"Không tìm thấy homestay với tên: {homestay_name}")
        return

    print(f"Uploading images for: {homestay_name}")
    image_urls = []
    for img_file in sorted(os.listdir(image_dir)):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
            img_path = os.path.join(image_dir, img_file)
            result = cloudinary.uploader.upload(img_path, folder=f"homestay/{homestay_name}")
            image_urls.append(result['secure_url'])
            print(f"  Uploaded: {img_file} -> {result['secure_url']}")

    # Cập nhật vào homestay
    homestay['image_urls'] = image_urls

    # Lưu lại file json
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(homestays, f, ensure_ascii=False, indent=2)
    print("Đã cập nhật xong homestays.json!")

if __name__ == "__main__":
    homestay_name = input("Nhập tên homestay: ")
    image_dir = r'C:\\Users\\Xpeedent\\Desktop\\Image for homestay\\Image for homestay\\Space Homestay'
    json_path = 'data/homestays.json'
    upload_images_for_homestay(homestay_name, image_dir, json_path)

Uploading images for: Space Homestay
  Uploaded: 1.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136968/homestay/Space%20Homestay/uymfopad9q6xtwgsshks.jpg
  Uploaded: 10.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136969/homestay/Space%20Homestay/uz7o5x14kjeldoemgfa7.jpg
  Uploaded: 2.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136970/homestay/Space%20Homestay/aauks9wp7dcfq0l557fj.jpg
  Uploaded: 3.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136971/homestay/Space%20Homestay/rd7g7lahq0c3rrdp3oey.jpg
  Uploaded: 4.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136972/homestay/Space%20Homestay/vr7ouaxj2mg60ysqklrd.jpg
  Uploaded: 5.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136973/homestay/Space%20Homestay/ntq7yx56qar3i1d5b1b7.jpg
  Uploaded: 6.jpg -> https://res.cloudinary.com/drsfudaav/image/upload/v1748136973/homestay/Space%20Homestay/znwsuddvdhwmgpyb2zev.jpg
  Uploaded: 7.jpg -> htt